In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [2]:
# Define folder paths for each dog breed
folder_paths = {
    'Bengal': r"C:\Users\joshd\OneDrive\Documents\PBL\Cat Breeds Folder\Bengal",
    'Bombay': r"C:\Users\joshd\OneDrive\Documents\PBL\Cat Breeds Folder\Bombay",
    'Persian': r"C:\Users\joshd\OneDrive\Documents\PBL\Cat Breeds Folder\Persian",
    'Siamese': r"C:\Users\joshd\OneDrive\Documents\PBL\Cat Breeds Folder\Siamese"
}

In [3]:
# Function to load images from a folder and assign labels, resize and normalize them
def load_images_from_folder(folder, label, target_size=(256, 256)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, target_size)  # Resize the image
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
            img = img.astype('float32') / 255.0  # Normalize pixel values to [0, 1]
            images.append(img)
            labels.append(label)
    return images, labels

In [4]:
# Load images and labels for each breed
data = []
labels = []
for breed, path in folder_paths.items():
    breed_images, breed_labels = load_images_from_folder(path, breed)
    data.extend(breed_images)
    labels.extend(breed_labels)

In [5]:
# Convert lists to numpy arrays
data = np.array(data)
labels = np.array(labels)

In [6]:
# Convert labels to integer indices
label_dict = {label: idx for idx, label in enumerate(np.unique(labels))}
labels = np.array([label_dict[label] for label in labels])

In [7]:
# Split data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)

In [8]:
# Print shapes for verification
print(train_data.shape, test_data.shape, train_labels.shape, test_labels.shape)

(8974, 256, 256, 3) (2244, 256, 256, 3) (8974,) (2244,)


In [9]:
# Build and compile the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(len(np.unique(labels)), activation='softmax')  # Adjust the output layer to match the number of breeds
])

C:\Users\joshd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
# Train the model
model.fit(train_data, train_labels, epochs=70, validation_data=(test_data, test_labels), batch_size=32)

Epoch 1/70
281/281 ━━━━━━━━━━━━━━━━━━━━ 270s 920ms/step - accuracy: 0.3999 - loss: 1.2398 - val_accuracy: 0.5530 - val_loss: 1.0123
Epoch 2/70
281/281 ━━━━━━━━━━━━━━━━━━━━ 184s 656ms/step - accuracy: 0.5942 - loss: 0.9219 - val_accuracy: 0.6190 - val_loss: 0.9149
Epoch 3/70
281/281 ━━━━━━━━━━━━━━━━━━━━ 234s 832ms/step - accuracy: 0.6911 - loss: 0.7529 - val_accuracy: 0.6791 - val_loss: 0.7905
Epoch 4/70
281/281 ━━━━━━━━━━━━━━━━━━━━ 242s 862ms/step - accuracy: 0.7623 - loss: 0.5988 - val_accuracy: 0.6894 - val_loss: 0.7767
Epoch 5/70
281/281 ━━━━━━━━━━━━━━━━━━━━ 241s 856ms/step - accuracy: 0.8344 - loss: 0.4105 - val_accuracy: 0.6698 - val_loss: 1.0154
Epoch 6/70
281/281 ━━━━━━━━━━━━━━━━━━━━ 242s 859ms/step - accuracy: 0.9082 - loss: 0.2534 - val_accuracy: 0.6653 - val_loss: 1.1974
Epoch 7/70
281/281 ━━━━━━━━━━━━━━━━━━━━ 241s 857ms/step - accuracy: 0.9546 - loss: 0.1420 - val_accuracy: 0.6635 - val_loss: 1.6696
Epoch 8/70
281/281 ━━━━━━━━━━━━━━━━━━━━ 242s 860ms/step - accuracy: 0.9714 -

In [12]:
# Print model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         2,052 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 39,265,358 (149.79 MB)

 Trainable params: 13,088,452 (49.93 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 26,176,906 (99.86 MB)

In [13]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_data, test_labels)
print(f"Test Accuracy: {test_acc:.3f}")  # Formatted to show accuracy as a percentage

71/71 ━━━━━━━━━━━━━━━━━━━━ 9s 128ms/step - accuracy: 0.6926 - loss: 5.0325
Test Accuracy: 0.668


In [14]:
# Define the file path to save the model
model_path = r"C:\Users\joshd\OneDrive\Documents\PBL\Cat Breeds Folder\cat_classification.h5"

In [15]:
# Save the model
model.save(model_path)
print(f"Model saved successfully at: {model_path}")

Model saved successfully at: C:\Users\joshd\OneDrive\Documents\PBL\Cat Breeds Folder\cat_classification.h5
